In [1]:
import pandas as pd
from helpers import predictor as pf

C:\Users\caiog\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\thefuzz\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


Setup Complete


In [ ]:
bankroll = 900
season = 2024

leagues = [
    {
        "league": "laliga",
        "country": "spain",
    },
    {
        "league": "j1-league",
        "country": "japan",
    },
    {
        "league": "bundesliga",
        "country": "germany",
    },
    # {
    #     "league": "mls",
    #     "country": "usa",
    # },
]

In [3]:
bets = []

for league_dict in leagues:
    league = league_dict["league"]
    country = league_dict["country"]

    print(f"\n{league.upper()}\n")

    model_data = pf.load_model_data(league)

    pipeline = model_data["pipeline"]
    best_model_name = model_data["best_model_name"]
    features = model_data["filtered_cols"]
    min_odds = model_data["min_odds"]
    min_games_played = model_data["min_games_played"]
    min_games_played_at = model_data["min_games_played_at"]
    strategy = model_data["strategy"]
    default_bankroll_pct = model_data["default_bankroll_pct"]
    default_value = model_data["default_value"]

    print("Features:", features)
    print("Min odds:", min_odds)
    display(pipeline)

    season_games, teams, teams_elo = pf.get_season_games(league, season)

    next_games = pf.get_next_games(country, league)

    data_model = []
    for game in next_games:
        stats = pf.get_next_games_stats(game, season, min_games_played, min_games_played_at, season_games, teams_elo, teams)
        
        if stats:
            data_model.append(stats)

    if not data_model:
        continue

    data_df = pd.DataFrame(data_model)

    predictions_df = pf.predict_next_games(pipeline, data_df, features)

    league_bets = pf.get_bets(predictions_df, min_odds, bankroll, strategy, default_value, default_bankroll_pct)

    bets.append({
        "league": league,
        "bets": league_bets
    })


LALIGA

Features: ['home_points_pct', 'home_win_pct', 'home_draw_pct', 'home_loss_pct', 'home_points_pct_last_games', 'home_win_pct_last_games', 'home_draw_pct_last_games', 'home_loss_pct_last_games', 'home_home_win_pct', 'home_home_draw_pct', 'home_home_loss_pct', 'home_team_score', 'home_opp_score', 'home_home_team_score', 'home_home_opp_score', 'away_points_pct', 'away_win_pct', 'away_draw_pct', 'away_loss_pct', 'away_points_pct_last_games', 'away_win_pct_last_games', 'away_draw_pct_last_games', 'away_loss_pct_last_games', 'away_away_win_pct', 'away_away_draw_pct', 'away_away_loss_pct', 'away_team_score', 'away_opp_score', 'away_away_opp_score', 'away_away_team_score', 'home_odds', 'away_odds', 'draw_odds', 'home_elo', 'away_elo']
Min odds: 2.2


Pipeline(steps=[('model',
                 VotingClassifier(estimators=[('svm',
                                               SVC(probability=True,
                                                   random_state=0)),
                                              ('random_forest_default',
                                               RandomForestClassifier(random_state=0)),
                                              ('gradient_boosting',
                                               GradientBoostingClassifier(random_state=0))],
                                  voting='soft'))])

Generating teams ELOs...


100%|██████████| 4697/4697 [00:00<00:00, 7900.86it/s]


Successfully generated teams ELOs.

J1-LEAGUE

Features: ['home_points_pct', 'home_win_pct', 'home_draw_pct', 'home_loss_pct', 'home_points_pct_last_games', 'home_win_pct_last_games', 'home_draw_pct_last_games', 'home_loss_pct_last_games', 'home_home_win_pct', 'home_home_draw_pct', 'home_home_loss_pct', 'home_team_score', 'home_opp_score', 'home_home_team_score', 'home_home_opp_score', 'away_points_pct', 'away_win_pct', 'away_draw_pct', 'away_loss_pct', 'away_points_pct_last_games', 'away_win_pct_last_games', 'away_draw_pct_last_games', 'away_loss_pct_last_games', 'away_away_win_pct', 'away_away_draw_pct', 'away_away_loss_pct', 'away_team_score', 'away_opp_score', 'away_away_opp_score', 'away_away_team_score', 'home_odds', 'away_odds', 'draw_odds', 'home_elo', 'away_elo']
Min odds: 2.8


...

Successfully generated teams ELOs.

BUNDESLIGA

Features: ['home_points_pct', 'home_points_pct_last_games', 'home_win_pct_last_games', 'home_draw_pct_last_games', 'home_loss_pct_last_games', 'home_team_score', 'home_opp_score', 'home_home_team_score', 'home_home_opp_score', 'away_points_pct', 'away_points_pct_last_games', 'away_win_pct_last_games', 'away_draw_pct_last_games', 'away_loss_pct_last_games', 'away_team_score', 'away_opp_score', 'away_away_opp_score', 'away_away_team_score', 'home_odds', 'away_odds', 'draw_odds', 'home_elo', 'away_elo']
Min odds: 2.9


Pipeline(steps=[('model',
                 VotingClassifier(estimators=[('naive_bayes', GaussianNB()),
                                              ('logistic_regression',
                                               LogisticRegression(random_state=0)),
                                              ('svm',
                                               SVC(probability=True,
                                                   random_state=0))],
                                  voting='soft'))])

Generating teams ELOs...


100%|██████████| 3776/3776 [00:00<00:00, 8745.97it/s]


Successfully generated teams ELOs.


In [4]:
for league in bets:
    print(f"\n{league['league'].upper()}\n")

    for bet in league['bets']:
        print(bet)


LALIGA

Barcelona (1812.72) x (1410.38) Las Palmas: $1 on Barcelona @ 1.14
Alavés (1452.49) x (1479.5) Leganés: $1 on Alavés @ 1.86
Espanyol (1423.98) x (1508.91) Celta Vigo: $1 on Espanyol @ 3.25
Valladolid (1400.03) x (1723.18) Atlético Madrid: $1 on Atlético Madrid @ 1.54
Villarreal (1632.85) x (1634.98) Girona: $1 on Villarreal @ 1.86
Real Madrid (1836.4) x (1473.47) Getafe: $1 on Real Madrid @ 1.33
Rayo Vallecano (1459.37) x (1633.96) Athletic Club: $1 on Athletic Club @ 2.65
Real Sociedad (1602.92) x (1577.63) Betis: $1 on Real Sociedad @ 1.74
Sevilla (1512.52) x (1533.62) Osasuna: $1 on Sevilla @ 2.13
Mallorca (1512.42) x (1812.72) Barcelona: $1 on Barcelona @ 1.66
Athletic Club (1633.96) x (1836.4) Real Madrid: $1 on Real Madrid @ 2.11
Celta Vigo (1508.91) x (1512.42) Mallorca: $1 on Celta Vigo @ 1.92

J1-LEAGUE

Albirex Niigata (1465.63) x (1558.06) Gamba Osaka: $45.0 on Gamba Osaka @ 2.82
Avispa Fukuoka (1514.24) x (1564.84) Urawa Reds: $45.0 on Urawa Reds @ 2.65
Cerezo Osak